In [10]:
from pprint import pprint
import json

name = 'second-batch';

with open('{}.json'.format(name)) as f:
    data = json.load(f)['logs']
    
with open('images/_airlines.json') as f:
    airlines = json.load(f)

with open('images/_states.json') as f:
    states = json.load(f)

In [11]:
from collections import OrderedDict 
from copy import deepcopy

rows = []

for p in data.items():
    user = p[0]
    d = OrderedDict({
        'id': user
    })
    
    item = p[1]
    
    if not 'done' in item:
        print('ignoring {}'.format(p[0]))
        continue
    
    d['userAgent'] = item['userAgent']
    d['isMobile'] = item['isMobile']
    
    cond = item['cond']
    
    errorAirline = cond['errorAirline']
    seqAirline = cond['seqAirline']
    errorStates = cond['errorStates']
    seqStates = cond['seqStates']
    focusAirline = cond['focusAirline']
    focusState = cond['focusState']
    
    for k, v in cond.items():
        d[k] = v
        
    howManyIndex = 0
    for k, v in item['approximateVis_airline'].items():
        if k == 'done':
            continue
        t = v['type']
        
        if not 'confidence' in v:
            raise Exception(user, v)
        
        approx = airlines["{}_{}".format(errorAirline, seqAirline)]
        
        prefix = 'approx.airline.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusAirline]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
    
    howManyIndex = 0
    for k, v in item['approximateVis_states'].items():
        if k == 'done':
            continue
        t = v['type']
        
        if not 'confidence' in v:
            raise Exception(user, v)
        
        approx = states["{}_{}".format(errorStates, seqStates)]
        
        prefix = 'approx.states.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusState]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
     
    def jaccard_similarity(x,y):
        intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
        union_cardinality = len(set.union(*[set(x), set(y)]))
        return intersection_cardinality/float(union_cardinality)
    
    # precise
            
    howManyIndex = 0
    for k, v in item['preciseVis_airline'].items():
        if k == 'done':
            continue
        t = v['type']
        
        if not 'confidence' in v:
            raise Exception(user, v)
        
        precise = airlines["{}_{}".format(1, 1)]
        approx = airlines["{}_{}".format(errorAirline, seqAirline)]
        
        prefix = 'precise.airline.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusAirline]
            d[p + 'precise'] = precise[focusAirline]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'precise'] = precise[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
            
        if (t == 'DidYouNotice'):
            p = prefix + 'didYouNotice.'
            d[p + 'answer'] = v['answer'] == 'yes'
            d[p + 'true'] = errorAirline > 1
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'SelectAll'):
            v = deepcopy(v)
            p = prefix + 'SelectAll.'
            d[p + 'confidence'] = v['confidence']
            
            del v['confidence']
            del v['type']
            
            answers = v.keys()
            prec = [k for k,v in precise.items() if v]
            appr = [k for k,v in approx.items() if v]
            d[p + 'answer'] = ','.join(answers)
            d[p + 'approx'] = ','.join(appr)
            d[p + 'precise'] = ','.join(prec)
            d[p + 'jaccard_answer_precise'] = jaccard_similarity(answers, prec)
            d[p + 'jaccard_approx_precise'] = jaccard_similarity(appr, prec)
            d[p + 'jaccard_approx_answer'] = jaccard_similarity(appr, answers)
            
    howManyIndex = 0
    for k, v in item['preciseVis_states'].items():
        if k == 'done':
            continue
        t = v['type']
        
        if not 'confidence' in v:
            raise Exception(user, v)
        
        precise = states["{}_{}".format(1, 1)]
        approx = states["{}_{}".format(errorStates, seqStates)]
        
        prefix = 'precise.states.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusState]
            d[p + 'precise'] = precise[focusState]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'precise'] = precise[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
            
        if (t == 'DidYouNotice'):
            p = prefix + 'didYouNotice.'
            d[p + 'answer'] = v['answer'] == 'yes'
            d[p + 'true'] = errorStates > 1
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'SelectAll'):
            v = deepcopy(v)
            p = prefix + 'SelectAll.'
            d[p + 'confidence'] = v['confidence']
            
            del v['confidence']
            del v['type']
            
            answers = v.keys()
            prec = [k for k,v in precise.items() if v]
            appr = [k for k,v in approx.items() if v]
            d[p + 'answer'] = ','.join(answers)
            d[p + 'approx'] = ','.join(appr)
            d[p + 'precise'] = ','.join(prec)
            d[p + 'jaccard_answer_precise'] = jaccard_similarity(answers, prec)
            d[p + 'jaccard_approx_precise'] = jaccard_similarity(appr, prec)
            d[p + 'jaccard_approx_answer'] = jaccard_similarity(appr, answers)
    
    for k, v in item['timing'].items():
         d['timing.' + k] = v
            
    for k, v in item['demographics'].items():
         d['demographics.' + k] = v
    
    rows.append(d)

pprint(rows[0])

ignoring 76632955
OrderedDict([('id', '87912012'),
             ('userAgent',
              'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
              '(KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'),
             ('isMobile', False),
             ('errorAirline', 200),
             ('seqAirline', 14),
             ('secondCondition', 'airline'),
             ('studyCondition', 'onlyNew'),
             ('firstCondition', 'states'),
             ('focusState', 'FL'),
             ('focusAirline', 'AA'),
             ('errorStates', 1),
             ('seqStates', 1),
             ('approx.airline.howManyCompare.0.answer', 2000),
             ('approx.airline.howManyCompare.0.data', 'VX'),
             ('approx.airline.howManyCompare.0.approx', 200),
             ('approx.airline.howManyCompare.0.confidence', 'weak_disagree'),
             ('approx.airline.howManyCompare.1.answer', 600),
             ('approx.airline.howManyCompare.1.data', 'DL'),
             (

In [12]:
import csv

with open('{}.csv'.format(name), 'w') as f:
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    
    for row in rows:
        writer.writerow(row)

import pandas as pd

pd.read_csv('{}.csv'.format(name))

,id,userAgent,isMobile,errorAirline,seqAirline,secondCondition,studyCondition,firstCondition,focusState,focusAirline,...,timing.approximate_first_html,timing.approximate_questions_first_html,timing.approximate_questions_second_html,demographics.problems,demographics.improved,demographics.gender,demographics.age,demographics.troubles,demographics.cheat,demographics.familiar
0,87912012,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,200,14,airline,onlyNew,states,FL,AA,...,24134,41039,50485.0,no,none thank you,male,44,no,no,expert
1,50263296,Mozilla/5.0 (Windows NT 6.3; WOW64; rv:50.0) G...,False,100,5,airline,difference,states,FL,AA,...,62632,60470,46440.0,no,no,female,43,no,no,expert
2,76648162,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,False,200,20,states,both,airline,NY,AA,...,52391,75826,84506.0,No technical problems,No comments,Male,44,no,no,familiar
3,60755967,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) G...,False,1000,12,states,difference,airline,NY,AA,...,17904,21317,NaN,NaN,NaN,female,28,no,no,familiar
4,51665387,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,500,3,states,both,airline,MA,B6,...,35977,47007,44026.0,no,no,Male,39,no,no,familiar
5,2893057,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,False,100,5,states,both,airline,FL,AA,...,34029,37180,28645.0,no,longer timer,Male,32,no,no,none
6,25917069,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,200,15,states,both,airline,FL,DL,...,866229,52130,18845.0,-,-,male,29,no,no,expert
7,28653725,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,200,14,states,difference,airline,FL,AA,...,34604,39807,36402.0,none,none,male,30,no,no,familiar
8,30619584,Mozilla/5.0 (Windows NT 10.0; WOW64; rv:50.0) ...,False,1000,16,states,both,airline,FL,AA,...,47978,88097,79659.0,no,no,female,28,no,no,some
